In [1]:
import spacy
import os

nlp = spacy.load("ru_core_news_md")
directory = "/home/datadisk/jupyter-ostrovskij_av/Ostrovskij_av/txt_vec/"
processed_files = {}

def process_text(file_name, text):
    global processed_files
    doc = nlp(text) # Обрабатываем текст с помощью Spacy
    filtered_words = [token.text for token in doc if not token.is_stop] # Фильтруем стоп-слова
    filtered_text = ' '.join(filtered_words) # Формируем новый текст без стоп-слов
    doc_filtered = nlp(filtered_text) # Обрабатываем отфильтрованный текст
    sentences = list(doc_filte # Разбиваем текст на предложения
                     
    if not sentences:  # Если предложения не найдены, выводим сообщение
        print(f"Предложения не найдены в файле: {file_name}")
        return
                     
    # Создаем словарь с предложениями и их векторами
    sentence_vectors = { 
        i + 1: (sentence.text.strip(), sentence.vector)
        for i, sentence in enumerate(sentences)
    }
    # Сохраняем оригинальный текст, отфильтрованный текст и вектора предложений
    processed_files[file_name] = {
        "original_text": text,
        "filtered_text": filtered_text,
        "sentence_vectors": sentence_vectors,
    }

# Обработка всех текстовых файлов в директории
for file_name in os.listdir(directory):
    if file_name.endswith(".txt"): 
        file_path = os.path.join(directory, file_name)
        with open(file_path, "r", encoding="utf-8") as file:
            text = file.read()
            process_text(file_name, text)

def get_file_sentences(file_name, sentence_number):
    if file_name in processed_files: # Проверяем, что файл был обработан
        file_data = processed_files[file_name]
        sentence_vectors = file_data["sentence_vectors"]
        if sentence_number in sentence_vectors: # Проверяем, что предложение существует
            sentence_text, sentence_vector = sentence_vectors[sentence_number]
            print(f"Файл: {file_name}")
            print(f"Предложение {sentence_number}: {sentence_text}")
            print(f"Вектор предложения: {sentence_vector}")
        else:
            print(f"Предложение с номером {sentence_number} не найдено в файле {file_name}.")
    else:
        print(f"Файл {file_name} не найден в обработанных данных.")


get_file_sentences("50_11_2011.txt", 5)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
import numpy as np

def cosine_distance(vec1, vec2):
    dot_product = np.dot(vec1, vec2) # Скалярное произведение векторов
    norm1 = np.linalg.norm(vec1) # Норма (длина) первого вектора
    norm2 = np.linalg.norm(vec2) # Норма (длина) второго вектора
    if norm1 == 0 or norm2 == 0: # Проверка на нулевой вектор
        return 1.0  # Если один из векторов нулевой, расстояние максимальное
    return 1 - (dot_product / (norm1 * norm2)) # Косинусовое расстояние

def find_closest_sentence(query):
    global processed_files
    query_doc = nlp(query) # Обрабатываем поисковый запрос с помощью Spacy
    query_vector = query_doc.vector # Получаем вектор запроса

    # Если не удалось создать вектор
    if query_vector is None or not query_vector.any(): 
        print("Не удалось создать вектор для поискового запроса.")
        return

    closest_sentence = None
    closest_distance = float("inf") # Изначально расстояние максимально
    closest_file = None
    closest_sentence_number = None

    for file_name, file_data in processed_files.items(): # Проходим по обработанным файлам
        for sentence_number, (sentence_text, sentence_vector) in file_data["sentence_vectors"].items():
            if sentence_vector is not None and sentence_vector.any(): # Проверяем, что вектор предложения существует
                distance = cosine_distance(query_vector, sentence_vector) # Вычисляем косинусовое расстояние
                if distance < closest_distance: # Если найдено более близкое предложение
                    closest_distance = distance
                    closest_sentence = sentence_text
                    closest_file = file_name
                    closest_sentence_number = sentence_number

    if closest_sentence: # Если найдено ближайшее предложение
        print(f"Ближайшее предложение найдено в файле: {closest_file}")
        print(f"Номер предложения: {closest_sentence_number}")
        print(f"Текст предложения: {closest_sentence}")
        print(f"Косинусовое расстояние: {closest_distance}")
    else:
        print("Не удалось найти подходящее предложение.")
        
query = '''Примечания 
    1 Техническое обеспечение состоит из персональных компьютеров ,
    систем хранения , рабочего потока обнаружения, сети коммуникационных приложений '''
find_closest_sentence(query)